In [14]:
import gmaps
import pandas as pd
import numpy as np
import requests
import json

from config import gkey

# Configure gmaps
gmaps.configure(api_key=gkey)
gmaps.figure()

Figure(layout=FigureLayout(height='420px'))

In [18]:
crashes = pd.read_csv("Resources/Traffic_Crashes_-_Crashes.csv", low_memory=False)
crashes.head()

,RD_NO,CRASH_DATE_EST_I,CRASH_DATE,POSTED_SPEED_LIMIT,TRAFFIC_CONTROL_DEVICE,DEVICE_CONDITION,WEATHER_CONDITION,LIGHTING_CONDITION,FIRST_CRASH_TYPE,TRAFFICWAY_TYPE,...,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE,LOCATION
0,JC190956,NaN,03/19/2019 12:10:00 AM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",REAR END,NOT DIVIDED,...,0.0,0.0,2.0,0.0,0,3,3,41.946678,-87.712656,POINT (-87.712656238768 41.946677671109)
1,JC190951,NaN,03/19/2019 12:05:00 AM,25,NO CONTROLS,NO CONTROLS,CLEAR,"DARKNESS, LIGHTED ROAD",PARKED MOTOR VEHICLE,ONE-WAY,...,0.0,0.0,2.0,0.0,0,3,3,41.882281,-87.750095,POINT (-87.750094540628 41.882281005383)
2,JC190946,NaN,03/18/2019 11:30:00 PM,45,TRAFFIC SIGNAL,NO CONTROLS,CLEAR,DARKNESS,TURNING,ONE-WAY,...,0.0,0.0,2.0,0.0,23,2,3,41.794195,-87.631926,POINT (-87.63192639781 41.794195272036)
3,JC190967,NaN,03/18/2019 11:15:00 PM,30,STOP SIGN/FLASHER,FUNCTIONING PROPERLY,CLEAR,DARKNESS,REAR END,NOT DIVIDED,...,0.0,0.0,3.0,0.0,23,2,3,41.881892,-87.747898,POINT (-87.747898416744 41.881892308511)
4,JC190923,NaN,03/18/2019 10:45:00 PM,30,TRAFFIC SIGNAL,FUNCTIONING PROPERLY,CLEAR,"DARKNESS, LIGHTED ROAD",TURNING,DIVIDED - W/MEDIAN (NOT RAISED),...,0.0,0.0,3.0,0.0,22,2,3,41.735964,-87.643682,POINT (-87.643681724352 41.735963647349)


In [3]:
# crashes["CRASH_DATE"].min()
crashes.describe()

,POSTED_SPEED_LIMIT,LANE_CNT,STREET_NO,BEAT_OF_OCCURRENCE,NUM_UNITS,INJURIES_TOTAL,INJURIES_FATAL,INJURIES_INCAPACITATING,INJURIES_NON_INCAPACITATING,INJURIES_REPORTED_NOT_EVIDENT,INJURIES_NO_INDICATION,INJURIES_UNKNOWN,CRASH_HOUR,CRASH_DAY_OF_WEEK,CRASH_MONTH,LATITUDE,LONGITUDE
count,279171.000000,1.928170e+05,279171.000000,279167.000000,278651.000000,278130.000000,278130.000000,278130.000000,278130.000000,278130.000000,278130.000000,278130.0,279171.000000,279171.000000,279171.000000,278036.000000,278036.000000
mean,28.165845,1.367694e+01,3576.130411,1244.112746,2.020502,0.154086,0.000877,0.016888,0.082670,0.053651,2.030842,0.0,13.186814,4.126392,6.729818,41.857757,-87.671177
std,6.900461,3.008778e+03,2936.758157,712.925764,0.405019,0.507382,0.032279,0.152116,0.367828,0.295434,1.157985,0.0,5.419958,1.966082,3.593833,0.365009,0.745832
min,0.000000,0.000000e+00,0.000000,111.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,1.000000,1.000000,0.000000,-87.933994
25%,30.000000,2.000000e+00,1161.000000,712.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.0,9.000000,2.000000,3.000000,41.787254,-87.719934
50%,30.000000,2.000000e+00,3100.000000,1214.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.0,14.000000,4.000000,7.000000,41.878086,-87.671593
75%,30.000000,4.000000e+00,5501.000000,1824.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000,0.0,17.000000,6.000000,10.000000,41.924699,-87.632558
max,99.000000,1.191625e+06,451100.000000,2535.000000,18.000000,21.000000,3.000000,7.000000,21.000000,10.000000,61.000000,0.0,23.000000,7.000000,12.000000,42.022780,0.000000


In [4]:
crash_location = crashes.loc[:,"LATITUDE":"LONGITUDE"]
crash_location["LATITUDE"].dropna(how='any', inplace=True)
crash_location["LONGITUDE"].dropna(how='any', inplace=True)
crash_location.rename(columns={"LATITUDE": "Lat", "LONGITUDE":"Lng"}, inplace=True)
crash_location.head()

,Lat,Lng
0,41.946678,-87.712656
1,41.882281,-87.750095
2,41.794195,-87.631926
3,41.881892,-87.747898
4,41.735964,-87.643682


In [5]:
crash_location.describe()

,Lat,Lng
count,278036.000000,278036.000000
mean,41.857757,-87.671177
std,0.365009,0.745832
min,0.000000,-87.933994
25%,41.787254,-87.719934
50%,41.878086,-87.671593
75%,41.924699,-87.632558
max,42.022780,0.000000


In [15]:
# longitude between -80 and -100
# latitude between 30 and 50
target_url = (f"https://maps.googleapis.com/maps/api/geocode/json?address=Chicago&key={gkey}")
geo_data = requests.get(target_url).json()
print(json.dumps(geo_data, indent=4, sort_keys=True))

{
    "results": [
        {
            "address_components": [
                {
                    "long_name": "Chicago",
                    "short_name": "Chicago",
                    "types": [
                        "locality",
                        "political"
                    ]
                },
                {
                    "long_name": "Cook County",
                    "short_name": "Cook County",
                    "types": [
                        "administrative_area_level_2",
                        "political"
                    ]
                },
                {
                    "long_name": "Illinois",
                    "short_name": "IL",
                    "types": [
                        "administrative_area_level_1",
                        "political"
                    ]
                },
                {
                    "long_name": "United States",
                    "short_name": "US",
                    "types": [
 

In [16]:
lat_min = geo_data['results'][0]['geometry']['bounds']['southwest']['lat']
lat_max = geo_data['results'][0]['geometry']['bounds']['northeast']['lat']
lng_min = geo_data['results'][0]['geometry']['bounds']['southwest']['lng']
lng_max = geo_data['results'][0]['geometry']['bounds']['northeast']['lng']
print(f"Illinois spans latitude {lat_min} to {lat_max}.")
print(f"Illinois spans longitude {lng_min} to {lng_max}.")
print("We will need to discard any errant data that falls outside these bounds.")

Illinois spans latitude 41.6443349 to 42.023131.
Illinois spans longitude -87.9402669 to -87.52366099999999.
We will need to discard any errant data that falls outside these bounds.


In [8]:
i = 0
for index, row in crash_location.iterrows():
    if (crash_location["Lat"][index] < lat_min) | (crash_location["Lat"][index] > lat_max) | (np.isnan(crash_location["Lat"][index])) | (crash_location["Lng"][index] < lng_min) | (crash_location["Lng"][index] > lng_max) | (np.isnan(crash_location["Lng"][index])):
        print(f"Out of bounds: lat/lng {crash_location['Lat'][index]}, {crash_location['Lng'][index]}. Dropping record...")
        crash_location.drop(index, inplace=True)
        i += 1
print(f"Cleaning complete: {i} rows were dropped due to lat/lng outside bounds.")

Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bound

Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bound

Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bound

Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bound

Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bound

Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bound

Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bound

Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bounds: lat/lng nan, nan. Dropping record...
Out of bound

In [9]:
crash_location.describe()

,Lat,Lng
count,278016.000000,278016.000000
mean,41.860768,-87.677484
std,0.084798,0.057699
min,41.644670,-87.933994
25%,41.787269,-87.719936
50%,41.878097,-87.671603
75%,41.924702,-87.632561
max,42.022780,-87.524587


In [10]:
# Plot Heatmap

fig = gmaps.figure()
heatmap_layer = gmaps.heatmap_layer(crash_location, max_intensity=1000, point_radius=.01, dissipating=False)

heatmap_layer.dissipating = False
heatmap_layer.max_intensity = 1000
heatmap_layer.point_radius = .01

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
rl_cams = pd.read_csv("Resources/Red_Light_Camera_Locations.csv", low_memory=False)
rl_cams.head()

,INTERSECTION,FIRST APPROACH,SECOND APPROACH,THIRD APPROACH,GO LIVE DATE,LATITUDE,LONGITUDE,LOCATION
0,Pulaski-Diversey,SB,NB,NaN,05/15/2009,41.931791,-87.726979,"(41.931791, -87.726979)"
1,Pulaski-Foster,SB,WB,NaN,10/18/2007,41.975532,-87.728234,"(41.975532, -87.728234)"
2,Cicero-Fullerton,SB,WB,NaN,02/25/2006,41.924237,-87.746302,"(41.924237, -87.746302)"
3,Harlem-Belmont,NB,WB,NaN,06/18/2007,41.937997,-87.806746,"(41.937997, -87.806746)"
4,Narragansett-Fullerton,EB,WB,NaN,11/14/2008,41.923676,-87.785441,"(41.923676, -87.785441)"


In [12]:
rl_location = rl_cams.loc[:,"LATITUDE":"LONGITUDE"]
rl_location["LATITUDE"].dropna(how='any', inplace=True)
rl_location["LONGITUDE"].dropna(how='any', inplace=True)
rl_location.rename(columns={"LATITUDE": "Lat", "LONGITUDE":"Lng"}, inplace=True)
rl_location.head()

,Lat,Lng
0,41.931791,-87.726979
1,41.975532,-87.728234
2,41.924237,-87.746302
3,41.937997,-87.806746
4,41.923676,-87.785441


In [13]:
rl_intersection = rl_cams.loc[:,"INTERSECTION"]

rl_intersection_layer = gmaps.symbol_layer(
    rl_location, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Location: {intersection}" for intersection in rl_intersection]
)

fig = gmaps.figure()

fig.add_layer(heatmap_layer)
fig.add_layer(rl_intersection_layer)

fig 


Figure(layout=FigureLayout(height='420px'))